In [1]:
!pip install mahaNLP==0.5 gdown -q

In [2]:
!gdown --id 1VDwcsXYmh1uq8X7-8va25CP1JqL4xdv0

/opt/conda/lib/python3.10/site-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1VDwcsXYmh1uq8X7-8va25CP1JqL4xdv0
To: /kaggle/working/mahasent_train_new_part1.csv
100%|███████████████████████████████████████| 3.23M/3.23M [00:00<00:00, 139MB/s]


In [3]:
import warnings
warnings.simplefilter('ignore')

In [4]:
import mahaNLP
import pandas as pd
import torch
import random
import re
import gdown

In [5]:
# data preprocessing:
# preprocessing function, don't remove the punctuations at all.
import re
def remove_non_marathi_in_tweets(text):
    i= re.sub(r"http\S+", "", text)
    i= re.sub(r"pic\S+", "", i)
    i= re.sub(r"#", "", i)
    i= re.sub(r"lokmat\S+", "", i)
    i= re.sub(r"@\S+", "", i)
    i= re.sub(r"fb\S+", "", i)
    i= re.sub(r"twitter\S+", "", i)
    i= re.sub(r"bit\S+", "", i)    
#     i= re.sub(r"['\"!-./,;\[\]?]", "", i)  # during augmenting, keep the necessary punctuations [.?!'"] in the sentence.  
    i= re.sub(r"[\[\]]", "", i)    
    i= re.sub(r'[a-zA-Z0-9]', "", i)    
    i=re.split(r'[\s\n]+', i.strip())
    return(' '.join(i))

In [6]:
df = pd.read_csv('mahasent_train_new_part1.csv')
df.head()

,Unnamed: 0,tweet,label
0,0,ज्येष्ठ पत्रकार अनंत दीक्षित यांच्या निधनाचे वृत्त दु:खद आहे. चार दशकं त्यांनी आपल्या परखड लेखणीने पत्रकारितेत अमूल्य योगदान दिले. दीक्षित यांच्या मार्गदर्शनाखाली पत्रकारांची पिढी घडली. अनंत दीक्षित यांना भावपूर्ण श्रद्धांजली! \nआम्ही त्यांच्या परिवाराच्या दु:खात सहभागी आहोत. pic.twitter.com/s3gnQQLtpk\n,-1
1,1,सर्वोच्च न्यायालयाचे निर्देश डावलून पुणे पोलिसांनी प्रा. आनंद तेलतुंबडेंना अटक केली मात्र न्यायालयाने त्यांची अटक बेकायदेशीर ठरवून सरकारला जोरदार चपराक लगावली आहे. कायदा हातात घेणा-या पोलीस अधिका-यांवर तात्काळ कारवाई केली पाहिजे.\n#AnandTeltumbde\n,-1
2,2,उद्धव ठाकरेंनी भाजपासोबत युती करून शिवसैनिकांचा अवमान व विश्वासघात केला आहे फसवणूक झालेल्या शिवसैनिकांबद्दल मला सहानुभूती असून स्वार्थासाठी उद्धव ठाकरे भाजपसोबत गेले तरी मोदींचे सरकार घालवण्यातच महाराष्ट्राचे हित आहे हे जाणणारे शिवसैनिक मोदींच्या विरोधात काँग्रेसला मतदान करतील\n,-1
3,3,"आपला समाज खूप मोठा आहे. त्यात अनेक घटक अंतर्भूत आहेत. यातील दिव्यांग हा एक घटक, जो शारीरिकदृष्ट्या कमकुवत आहे, त्याच्याकडे समाजाचे दुर्लक्ष होतेय. त्यामुळे सरकारने अशा व्यक्तींसाठी उपक्रम राबवून त्यांना समाजात सन्मानाने जगता येईल, याकडे लक्ष देण्याची गरज आहे. pic.twitter.com/g0tbJyyTPX\n",1
4,4,बलात्काराचा बदला बलात्काराने घेतला पाहिजे हे म्हणणारे सावरकर आम्हाला मान्य नाहीत. द्वेष आणि तिरस्कार हिंदू धर्मातही अभिप्रेत कधीच नव्हता. आम्ही सावरकरांच्या विचारांचा विरोध करताच राहू त्यात व्यक्तीद्वेष नाही तर वैचारिक विरोध आहे. हा देश गांधींच्या विचारांनीच चालेल!\n,-1


In [7]:
df.drop(['Unnamed: 0'], inplace=True, axis=1)

In [8]:
for i in range(len(df)):
  df['tweet'][i] = remove_non_marathi_in_tweets(df['tweet'][i])

In [9]:
df = df[df['tweet']!='']
df.drop_duplicates(inplace=True)
df.dropna(inplace=True)
df = df.reset_index(drop=True)

In [10]:
df.head()

,tweet,label
0,ज्येष्ठ पत्रकार अनंत दीक्षित यांच्या निधनाचे वृत्त दु:खद आहे. चार दशकं त्यांनी आपल्या परखड लेखणीने पत्रकारितेत अमूल्य योगदान दिले. दीक्षित यांच्या मार्गदर्शनाखाली पत्रकारांची पिढी घडली. अनंत दीक्षित यांना भावपूर्ण श्रद्धांजली! आम्ही त्यांच्या परिवाराच्या दु:खात सहभागी आहोत.,-1
1,सर्वोच्च न्यायालयाचे निर्देश डावलून पुणे पोलिसांनी प्रा. आनंद तेलतुंबडेंना अटक केली मात्र न्यायालयाने त्यांची अटक बेकायदेशीर ठरवून सरकारला जोरदार चपराक लगावली आहे. कायदा हातात घेणा-या पोलीस अधिका-यांवर तात्काळ कारवाई केली पाहिजे.,-1
2,उद्धव ठाकरेंनी भाजपासोबत युती करून शिवसैनिकांचा अवमान व विश्वासघात केला आहे फसवणूक झालेल्या शिवसैनिकांबद्दल मला सहानुभूती असून स्वार्थासाठी उद्धव ठाकरे भाजपसोबत गेले तरी मोदींचे सरकार घालवण्यातच महाराष्ट्राचे हित आहे हे जाणणारे शिवसैनिक मोदींच्या विरोधात काँग्रेसला मतदान करतील,-1
3,"आपला समाज खूप मोठा आहे. त्यात अनेक घटक अंतर्भूत आहेत. यातील दिव्यांग हा एक घटक, जो शारीरिकदृष्ट्या कमकुवत आहे, त्याच्याकडे समाजाचे दुर्लक्ष होतेय. त्यामुळे सरकारने अशा व्यक्तींसाठी उपक्रम राबवून त्यांना समाजात सन्मानाने जगता येईल, याकडे लक्ष देण्याची गरज आहे.",1
4,बलात्काराचा बदला बलात्काराने घेतला पाहिजे हे म्हणणारे सावरकर आम्हाला मान्य नाहीत. द्वेष आणि तिरस्कार हिंदू धर्मातही अभिप्रेत कधीच नव्हता. आम्ही सावरकरांच्या विचारांचा विरोध करताच राहू त्यात व्यक्तीद्वेष नाही तर वैचारिक विरोध आहे. हा देश गांधींच्या विचारांनीच चालेल!,-1


In [11]:
from mahaNLP.tokenizer import Tokenize
tokenizer = Tokenize()
from mahaNLP.tagger import EntityRecognizer
ner_tagger = EntityRecognizer()
from mahaNLP.maskFill import MaskPredictor
mask_predictor = MaskPredictor()

In [12]:
list_augmented_sentences = []

def mask_NER_entities_in_text_parallel(text):
    # getting the NER tokens from the sentence
    df_ner = ner_tagger.getTokenLabels(text)

    # converting the NER tokens into a list, excluding those from the 'Other' category
    ner_tokens = list(df_ner.loc[df_ner['entity_group'] != 'Other', 'word'])
    # print(ner_tokens)

    

    for token in ner_tokens:
      text = text.replace(token, token+" ", 1)
    # print(text)

    for token in ner_tokens:
      if token not in text:
        ner_tokens.remove(token)

    # for token in ner_tokens:
    #   print(mask_predictor.predictMask(text.replace(token, '[MASK] ', 1))['token_str'][0])

    # replace each NER token with [MASK], get the predicted word, and store in a list
    masked_prediction_list = [
        mask_predictor.predictMask(text.replace(token, '[MASK] ', 1))['token_str'][0]
        if mask_predictor.predictMask(text.replace(token, '[MASK] ', 1))['token_str'][0] != token
        else mask_predictor.predictMask(text.replace(token, '[MASK] ', 1))['token_str'][1]
        for token in ner_tokens
    ]

    # split the sentence into words and replace each NER token with its corresponding predicted word
    words = text.split()
    for i, token in enumerate(words):
        if token in ner_tokens:
            words[i] = masked_prediction_list[ner_tokens.index(token)]

    # join the words back into a sentence and return
    return " ".join(words)

In [13]:
df['tweet'][800]

'आज माढा येथे पक्षाच्या कार्यकर्ता मेळाव्यात उपस्थित राहून उपस्थितांना मार्गदर्शन केले. कार्यकर्त्यांमध्ये प्रचंड उत्साह आहे. सरकारमध्ये असताना माढा येथे आम्ही एमआयडीसी सुरू केली होती. या निर्णयामुळे अनेक कामगार विविध कारखान्यांच्या माध्यमातून पोट भरत आहेत याचे समाधान वाटते.'

In [14]:
mask_NER_entities_in_text_parallel(df['tweet'][800])

'त्यांनी सोलापूर येथे पक्षाच्या कार्यकर्ता मेळाव्यात उपस्थित राहून उपस्थितांना मार्गदर्शन केले. कार्यकर्त्यांमध्ये प्रचंड उत्साह आहे. सरकारमध्ये असताना सोलापूर येथे आम्ही कारखाना सुरू केली होती. या निर्णयामुळे अनेक कामगार विविध कारखान्यांच्या माध्यमातून पोट भरत आहेत याचे समाधान वाटते.'

In [15]:
df_augmented = pd.DataFrame(columns=df.columns)

In [16]:
df_augmented

,tweet,label


In [17]:
from tqdm import tqdm
for i in tqdm(range(len(df))):
    try:
      row = df.iloc[i]
      j = re.sub(r'[\u200d]+', '', row['tweet'])
      tweet_augmented = mask_NER_entities_in_text_parallel(j)
      row_augmented = row.copy()
      row_augmented['tweet_augmented'] = tweet_augmented
      df_augmented = df_augmented.append(row_augmented, ignore_index=True)
    except Exception:
      continue

 38%|███▊      | 2289/6045 [2:04:59<1:42:16,  1.63s/it] 

Please mask your sentence first!


 75%|███████▍  | 4525/6045 [4:05:07<1:24:05,  3.32s/it] 

Please mask your sentence first!


100%|██████████| 6045/6045 [5:30:03<00:00,  3.28s/it]  


In [18]:
len(df_augmented)

6043

In [19]:
df_augmented

,tweet,label,tweet_augmented
0,ज्येष्ठ पत्रकार अनंत दीक्षित यांच्या निधनाचे वृत्त दु:खद आहे. चार दशकं त्यांनी आपल्या परखड लेखणीने पत्रकारितेत अमूल्य योगदान दिले. दीक्षित यांच्या मार्गदर्शनाखाली पत्रकारांची पिढी घडली. अनंत दीक्षित यांना भावपूर्ण श्रद्धांजली! आम्ही त्यांच्या परिवाराच्या दु:खात सहभागी आहोत.,-1,ज्येष्ठ पत्रकार नारायण दिक् यांच्या निधनाचे वृत्त दु:खद आहे. अनेक दशके त्यांनी आपल्या परखड लेखणीने पत्रकारितेत अमूल्य योगदान दिले. दिक् यांच्या मार्गदर्शनाखाली पत्रकारांची पिढी घडली. नारायण दिक् यांना भावपूर्ण श्रद्धांजली! आम्ही त्यांच्या परिवाराच्या दु:खात सहभागी आहोत.
1,सर्वोच्च न्यायालयाचे निर्देश डावलून पुणे पोलिसांनी प्रा. आनंद तेलतुंबडेंना अटक केली मात्र न्यायालयाने त्यांची अटक बेकायदेशीर ठरवून सरकारला जोरदार चपराक लगावली आहे. कायदा हातात घेणा-या पोलीस अधिका-यांवर तात्काळ कारवाई केली पाहिजे.,-1,उच्च न्यायालयाच्या निर्देश डावलून मुंबई पोलिसांनी आज मिलिंद यांना अटक केली मात्र न्यायालयाने त्यांची अटक बेकायदेशीर ठरवून सरकारला जोरदार चपराक लगावली आहे. कायदा हातात घेणा-या पोलीस अधिका-यांवर तात्काळ कारवाई केली पाहिजे.
2,उद्धव ठाकरेंनी भाजपासोबत युती करून शिवसैनिकांचा अवमान व विश्वासघात केला आहे फसवणूक झालेल्या शिवसैनिकांबद्दल मला सहानुभूती असून स्वार्थासाठी उद्धव ठाकरे भाजपसोबत गेले तरी मोदींचे सरकार घालवण्यातच महाराष्ट्राचे हित आहे हे जाणणारे शिवसैनिक मोदींच्या विरोधात काँग्रेसला मतदान करतील,-1,बाळासाहेब ##का ंनी भाजप युती करून शिवसैनिकांचा अवमान व विश्वासघात केला आहे फसवणूक झालेल्या शिवसैनिकांबद्दल मला सहानुभूती असून स्वार्थासाठी बाळासाहेब ##का बाहेर गेले तरी हे सरकार घालवण्यातच आपले हित आहे हे जाणणारे शिवसैनिक भाजपच्या विरोधात ##च मतदान करतील
3,"आपला समाज खूप मोठा आहे. त्यात अनेक घटक अंतर्भूत आहेत. यातील दिव्यांग हा एक घटक, जो शारीरिकदृष्ट्या कमकुवत आहे, त्याच्याकडे समाजाचे दुर्लक्ष होतेय. त्यामुळे सरकारने अशा व्यक्तींसाठी उपक्रम राबवून त्यांना समाजात सन्मानाने जगता येईल, याकडे लक्ष देण्याची गरज आहे.",1,"आपला समाज खूप मोठा आहे. त्यात अनेक घटक अंतर्भूत आहेत. यातील दिव्यांग हा एक घटक, जो शारीरिकदृष्ट्या कमकुवत आहे, त्याच्याकडे समाजाचे दुर्लक्ष होतेय. त्यामुळे सरकारने अशा व्यक्तींसाठी उपक्रम राबवून त्यांना समाजात सन्मानाने जगता येईल, याकडे लक्ष देण्याची गरज आहे."
4,बलात्काराचा बदला बलात्काराने घेतला पाहिजे हे म्हणणारे सावरकर आम्हाला मान्य नाहीत. द्वेष आणि तिरस्कार हिंदू धर्मातही अभिप्रेत कधीच नव्हता. आम्ही सावरकरांच्या विचारांचा विरोध करताच राहू त्यात व्यक्तीद्वेष नाही तर वैचारिक विरोध आहे. हा देश गांधींच्या विचारांनीच चालेल!,-1,बलात्काराचा बदला बलात्काराने घेतला पाहिजे हे म्हणणारे लोक आम्हाला मान्य नाहीत. द्वेष आणि तिरस्कार हिंदू धर्मातही अभिप्रेत कधीच नव्हता. आम्ही गांधींच्या विचारांचा विरोध करताच राहू त्यात व्यक्तीद्वेष नाही तर वैचारिक विरोध आहे. हा देश आमच्या विचारांनीच चालेल!
...,...,...,...
6038,"काँग्रेस नेते यांनी लिहिलेल्या विधानगाथा या पुस्तकाच्या प्रकाशन सोहळ्यास मुंबईतील विधानभवनात उपस्थित होतो. विधिमंडळात येणाऱ्या नव्या सदस्यांना मार्गदर्शक ठरेल, असे हे पुस्तक आहे.",0,"अशोक नेते यांनी लिहिलेल्या विधानगाथा या पुस्तकाच्या प्रकाशन सोहळ्यास मी पत्रकार उपस्थित होतो. विधिमंडळात येणाऱ्या नव्या सदस्यांना मार्गदर्शक ठरेल, असे हे पुस्तक आहे."
6039,"आपल्या अभंग, विचारवाणीतून व्यावहारिक सत्य मांडणारे, समता, विश्वबंधुत्वाचा संदेश देणारे जगद्गुरू, संतश्रेष्ठ श्री तुकाराम महाराज यांना 'तुकाराम बीज'निमित्त त्रिवार वंदन!",1,"आपल्या अभंग, विचारवाणीतून व्यावहारिक सत्य मांडणारे, समता, विश्वबंधुत्वाचा संदेश देणारे जगद्गुरू, संतश्रेष्ठ श्री ज्ञानेश्वर महाराज यांना 'तुकाराम बीज'निमित्त त्रिवार वंदन!"
6040,"त्याचबरोबर काजू व्यावसायिकांची मागील कालावधीतील व्हॅट परताव्याची थकीत रक्कमही परत करण्यात येणार आहे. काजू उत्पादक आणि प्रक्रिया उद्योजकांना मदत करण्याचे सरकारचे धोरण असून त्यांच्या अन्य मागण्यांबाबत सरकार सकारात्मक विचार करेल, असं स्पष्ट केलं.",1,"त्याचबरोबर काजू व्यावसायिकांची मागील कालावधीतील व्हॅट परताव्याची थकीत रक्कमही परत करण्यात येणार आहे. काजू उत्पादक आणि प्रक्रिया उद्योजकांना मदत करण्याचे सरकारचे धोरण असून त्यांच्या अन्य मागण्यांबाबत सरकार सकारात्मक विचार करेल, असं स्पष्ट केलं."
6041,सरकारने कर्जमाफी योजना लागू करुन सुमारे २ वर्षांचा काळ उलटून गेला आहे. मात्र अद्यापही कर्जमाफी मिळालेली नाही. शेतकऱ्यांना आगामी पीक घेण्यासाठी

In [20]:
df_augmented.drop(columns=['tweet'], axis=1, inplace=True)

In [21]:
df_augmented = df_augmented.rename(columns={'tweet_augmented': 'tweet'})
# df = pd.concat([df, df_augmented])
df = df.append(df_augmented, ignore_index = True)
df = df.drop_duplicates()
df.dropna(inplace=True)
df = df.reset_index(drop=True)
df.to_csv('mahasent_parallel_masking_ner_augmented.csv', index=False)

In [22]:
len(df)

11091